# Lab 3: Building a Simple AI Application

**Week 1 - GenAI Introduction & Fundamentals**

**Provided by:** ADC ENGINEERING & CONSULTING LTD

## Objectives

In this lab, you will:
- Design and build a complete AI-powered application
- Implement a text summarizer with multiple modes
- Add robust error handling and retry logic
- Implement streaming responses for better UX
- Create a command-line interface (CLI)
- Add logging and monitoring
- Deploy best practices for production-ready code

## Prerequisites

- Completed Labs 1 and 2
- Understanding of Python classes and async programming
- Familiarity with command-line applications
- OpenAI API key configured

## Setup and Installation

In [ ]:
# Install required packages
!pip install openai python-dotenv tiktoken rich click --quiet

In [ ]:
import os
import sys
import json
import time
from typing import List, Dict, Optional, Generator
from datetime import datetime
from pathlib import Path

from openai import OpenAI, OpenAIError, RateLimitError, APIError
from dotenv import load_dotenv
import tiktoken
from rich.console import Console
from rich.markdown import Markdown
from rich.progress import Progress, SpinnerColumn, TextColumn
from rich.panel import Panel
from rich.table import Table

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Initialize rich console for beautiful output
console = Console()

console.print("[bold green]✓[/bold green] Setup complete!")

## Part 1: Application Architecture

Before coding, let's design our text summarizer application with a clear architecture.

### Core Components

1. **SummarizerEngine**: Core logic for text summarization
2. **ConfigManager**: Handle configuration and settings
3. **ErrorHandler**: Robust error handling with retries
4. **StreamingManager**: Handle streaming responses
5. **CLI**: User interface
6. **Logger**: Track operations and errors

### Features to Implement

- Multiple summarization modes (concise, detailed, bullet points)
- File input/output support
- Streaming for long summaries
- Cost estimation
- Error handling with exponential backoff
- Logging and monitoring
- Configuration management

## Part 2: Configuration Management

Start by building a configuration system:

In [ ]:
class Config:
    """
    Configuration management for the summarizer application.
    """
    
    def __init__(self, config_file: Optional[str] = None):
        """
        Initialize configuration.
        
        Args:
            config_file: Path to JSON config file (optional)
        """
        # Default configuration
        self.defaults = {
            "model": "gpt-3.5-turbo",
            "temperature": 0.3,
            "max_tokens": 500,
            "max_retries": 3,
            "retry_delay": 1,
            "log_level": "INFO",
            "cost_warning_threshold": 0.10
        }
        
        self.config = self.defaults.copy()
        
        # Load from file if provided
        if config_file and Path(config_file).exists():
            self.load_from_file(config_file)
    
    def load_from_file(self, filepath: str):
        """Load configuration from JSON file."""
        try:
            with open(filepath, 'r') as f:
                file_config = json.load(f)
                self.config.update(file_config)
            console.print(f"[green]✓[/green] Configuration loaded from {filepath}")
        except Exception as e:
            console.print(f"[yellow]⚠[/yellow] Could not load config: {e}")
    
    def save_to_file(self, filepath: str):
        """Save current configuration to JSON file."""
        try:
            with open(filepath, 'w') as f:
                json.dump(self.config, f, indent=2)
            console.print(f"[green]✓[/green] Configuration saved to {filepath}")
        except Exception as e:
            console.print(f"[red]✗[/red] Could not save config: {e}")
    
    def get(self, key: str, default=None):
        """Get configuration value."""
        return self.config.get(key, default)
    
    def set(self, key: str, value):
        """Set configuration value."""
        self.config[key] = value
    
    def display(self):
        """Display current configuration."""
        table = Table(title="Current Configuration")
        table.add_column("Setting", style="cyan")
        table.add_column("Value", style="green")
        
        for key, value in self.config.items():
            table.add_row(key, str(value))
        
        console.print(table)

# Test configuration
config = Config()
config.display()

### Exercise 2.1: Enhanced Configuration

Add these features to the Config class:
1. Environment variable override support
2. Validation for configuration values
3. Configuration profiles (development, production)

In [ ]:
class EnhancedConfig(Config):
    """
    Enhanced configuration with validation and profiles.
    
    TODO: Implement:
    1. load_from_env() - Load from environment variables
    2. validate() - Validate configuration values
    3. load_profile(name) - Load a configuration profile
    """
    
    def __init__(self, config_file: Optional[str] = None, profile: str = "default"):
        super().__init__(config_file)
        self.profile = profile
        self.profiles = {
            "development": {
                "model": "gpt-3.5-turbo",
                "temperature": 0.5,
                "max_tokens": 300,
                "log_level": "DEBUG"
            },
            "production": {
                "model": "gpt-4",
                "temperature": 0.2,
                "max_tokens": 500,
                "log_level": "WARNING"
            }
        }
    
    # TODO: Implement enhanced methods
    
    pass

# Test
# config = EnhancedConfig(profile="production")
# config.validate()
# config.display()

## Part 3: Error Handling with Retry Logic

Implement robust error handling:

In [ ]:
class ErrorHandler:
    """
    Handle API errors with exponential backoff retry logic.
    """
    
    def __init__(self, max_retries: int = 3, base_delay: float = 1.0):
        """
        Initialize error handler.
        
        Args:
            max_retries: Maximum number of retry attempts
            base_delay: Base delay for exponential backoff (seconds)
        """
        self.max_retries = max_retries
        self.base_delay = base_delay
        self.retry_count = 0
    
    def handle_with_retry(self, func, *args, **kwargs):
        """
        Execute a function with retry logic.
        
        Args:
            func: Function to execute
            *args, **kwargs: Arguments for the function
        
        Returns:
            Function result or raises exception after max retries
        """
        last_exception = None
        
        for attempt in range(self.max_retries):
            try:
                result = func(*args, **kwargs)
                self.retry_count = 0  # Reset on success
                return result
                
            except RateLimitError as e:
                last_exception = e
                delay = self.base_delay * (2 ** attempt)  # Exponential backoff
                
                console.print(
                    f"[yellow]⚠[/yellow] Rate limit hit. "
                    f"Retrying in {delay}s... (Attempt {attempt + 1}/{self.max_retries})"
                )
                
                time.sleep(delay)
                
            except APIError as e:
                last_exception = e
                delay = self.base_delay
                
                console.print(
                    f"[yellow]⚠[/yellow] API error: {str(e)}. "
                    f"Retrying in {delay}s... (Attempt {attempt + 1}/{self.max_retries})"
                )
                
                time.sleep(delay)
                
            except OpenAIError as e:
                last_exception = e
                console.print(f"[red]✗[/red] OpenAI error: {str(e)}")
                break  # Don't retry for other OpenAI errors
                
            except Exception as e:
                last_exception = e
                console.print(f"[red]✗[/red] Unexpected error: {str(e)}")
                break
        
        # If we get here, all retries failed
        raise last_exception

# Test error handler
error_handler = ErrorHandler(max_retries=3)

def test_function():
    """Simulated function that might fail."""
    import random
    if random.random() < 0.5:
        raise APIError("Simulated API error")
    return "Success!"

# Uncomment to test
# try:
#     result = error_handler.handle_with_retry(test_function)
#     console.print(f"[green]✓[/green] {result}")
# except Exception as e:
#     console.print(f"[red]Failed after retries: {e}[/red]")

## Part 4: Core Summarization Engine

Build the main summarization engine:

In [ ]:
class SummarizerEngine:
    """
    Core text summarization engine.
    """
    
    def __init__(self, config: Config):
        """
        Initialize the summarizer.
        
        Args:
            config: Configuration object
        """
        self.config = config
        self.error_handler = ErrorHandler(
            max_retries=config.get("max_retries", 3),
            base_delay=config.get("retry_delay", 1)
        )
        self.token_counter = tiktoken.encoding_for_model(config.get("model"))
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in text."""
        return len(self.token_counter.encode(text))
    
    def estimate_cost(self, input_tokens: int, output_tokens: int) -> float:
        """Estimate cost for the operation."""
        model = self.config.get("model")
        pricing = {
            "gpt-3.5-turbo": {"input": 0.0005, "output": 0.0015},
            "gpt-4": {"input": 0.03, "output": 0.06}
        }
        
        if model not in pricing:
            return 0.0
        
        input_cost = (input_tokens / 1000) * pricing[model]["input"]
        output_cost = (output_tokens / 1000) * pricing[model]["output"]
        
        return input_cost + output_cost
    
    def summarize(
        self, 
        text: str, 
        mode: str = "concise",
        custom_instructions: Optional[str] = None
    ) -> Dict:
        """
        Summarize text with specified mode.
        
        Args:
            text: Text to summarize
            mode: Summarization mode ('concise', 'detailed', 'bullets', 'key_points')
            custom_instructions: Optional custom instructions
        
        Returns:
            Dictionary with summary, tokens, and cost
        """
        # Define prompts for different modes
        mode_prompts = {
            "concise": "Provide a concise summary in 2-3 sentences capturing the main points.",
            "detailed": "Provide a detailed summary covering all key points and supporting details.",
            "bullets": "Summarize as bullet points, highlighting key information in order of importance.",
            "key_points": "Extract and list the key points, insights, and conclusions.",
            "executive": "Provide an executive summary suitable for business leadership.",
            "technical": "Provide a technical summary preserving important terminology and concepts."
        }
        
        # Build the prompt
        if custom_instructions:
            instruction = custom_instructions
        else:
            instruction = mode_prompts.get(mode, mode_prompts["concise"])
        
        prompt = f"{instruction}\n\nText to summarize:\n{text}"
        
        # Count input tokens
        input_tokens = self.count_tokens(prompt)
        
        # Make API call with error handling
        def make_api_call():
            return client.chat.completions.create(
                model=self.config.get("model"),
                messages=[
                    {"role": "system", "content": "You are an expert summarizer. Provide clear, accurate summaries."},
                    {"role": "user", "content": prompt}
                ],
                temperature=self.config.get("temperature"),
                max_tokens=self.config.get("max_tokens")
            )
        
        response = self.error_handler.handle_with_retry(make_api_call)
        
        # Extract results
        summary = response.choices[0].message.content
        output_tokens = response.usage.completion_tokens
        total_tokens = response.usage.total_tokens
        
        # Calculate cost
        cost = self.estimate_cost(input_tokens, output_tokens)
        
        return {
            "summary": summary,
            "mode": mode,
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "total_tokens": total_tokens,
            "cost": cost,
            "model": self.config.get("model")
        }
    
    def summarize_stream(
        self,
        text: str,
        mode: str = "concise",
        custom_instructions: Optional[str] = None
    ) -> Generator[str, None, Dict]:
        """
        Summarize with streaming output.
        
        Yields:
            Chunks of the summary as they're generated
        
        Returns:
            Final metadata dictionary
        """
        # Build prompt (same as non-streaming)
        mode_prompts = {
            "concise": "Provide a concise summary in 2-3 sentences capturing the main points.",
            "detailed": "Provide a detailed summary covering all key points and supporting details.",
            "bullets": "Summarize as bullet points, highlighting key information.",
            "key_points": "Extract and list the key points, insights, and conclusions."
        }
        
        instruction = custom_instructions or mode_prompts.get(mode, mode_prompts["concise"])
        prompt = f"{instruction}\n\nText to summarize:\n{text}"
        
        # Count tokens
        input_tokens = self.count_tokens(prompt)
        
        # Make streaming API call
        def make_streaming_call():
            return client.chat.completions.create(
                model=self.config.get("model"),
                messages=[
                    {"role": "system", "content": "You are an expert summarizer."},
                    {"role": "user", "content": prompt}
                ],
                temperature=self.config.get("temperature"),
                max_tokens=self.config.get("max_tokens"),
                stream=True
            )
        
        stream = self.error_handler.handle_with_retry(make_streaming_call)
        
        full_summary = ""
        
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                full_summary += content
                yield content
        
        # Calculate final metrics
        output_tokens = self.count_tokens(full_summary)
        cost = self.estimate_cost(input_tokens, output_tokens)
        
        return {
            "summary": full_summary,
            "mode": mode,
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "total_tokens": input_tokens + output_tokens,
            "cost": cost,
            "model": self.config.get("model")
        }

# Test the summarizer
config = Config()
summarizer = SummarizerEngine(config)

# Sample text
sample_text = """
Machine learning is a method of data analysis that automates analytical model building. 
It is a branch of artificial intelligence based on the idea that systems can learn from data, 
identify patterns and make decisions with minimal human intervention. Machine learning algorithms 
are trained using large amounts of data and can improve their performance over time as they 
are exposed to more data. The three main types of machine learning are supervised learning, 
unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to 
train algorithms, unsupervised learning finds hidden patterns in unlabeled data, and 
reinforcement learning trains algorithms through trial and error using a reward system.
"""

console.print("[bold]Testing Summarizer Engine[/bold]\n")

# Test different modes
for mode in ["concise", "bullets", "key_points"]:
    console.print(f"[cyan]Mode: {mode}[/cyan]")
    result = summarizer.summarize(sample_text, mode=mode)
    
    console.print(Panel(result["summary"], title=f"{mode.upper()} Summary"))
    console.print(f"Tokens: {result['total_tokens']} | Cost: ${result['cost']:.6f}\n")

### Exercise 4.1: Batch Summarization

Implement a method to summarize multiple texts in batch:

In [ ]:
class BatchSummarizer(SummarizerEngine):
    """
    Summarizer with batch processing capabilities.
    
    TODO: Implement:
    1. summarize_batch(texts, mode) - Summarize multiple texts
    2. Progress tracking
    3. Aggregate cost reporting
    4. Error handling for individual items
    5. Save results to file
    """
    
    def __init__(self, config: Config):
        super().__init__(config)
        self.batch_results = []
    
    def summarize_batch(
        self,
        texts: List[str],
        mode: str = "concise",
        show_progress: bool = True
    ) -> List[Dict]:
        """
        Summarize multiple texts in batch.
        
        TODO: Implement with progress bar and error handling
        """
        pass
    
    def get_batch_report(self) -> Dict:
        """
        Generate report for batch operation.
        
        TODO: Calculate total tokens, costs, success rate
        """
        pass

# Test
# texts = ["Text 1...", "Text 2...", "Text 3..."]
# batch_summarizer = BatchSummarizer(config)
# results = batch_summarizer.summarize_batch(texts, mode="concise")
# print(batch_summarizer.get_batch_report())

## Part 5: File Operations

Add support for reading from and writing to files:

In [ ]:
class FileHandler:
    """
    Handle file input/output operations.
    """
    
    def __init__(self):
        self.supported_formats = ['.txt', '.md', '.json']
    
    def read_file(self, filepath: str) -> str:
        """
        Read content from file.
        
        Args:
            filepath: Path to input file
        
        Returns:
            File content as string
        """
        path = Path(filepath)
        
        if not path.exists():
            raise FileNotFoundError(f"File not found: {filepath}")
        
        if path.suffix not in self.supported_formats:
            raise ValueError(f"Unsupported file format: {path.suffix}")
        
        try:
            with open(path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            console.print(f"[green]✓[/green] Read {len(content)} characters from {filepath}")
            return content
            
        except Exception as e:
            console.print(f"[red]✗[/red] Error reading file: {e}")
            raise
    
    def write_file(self, filepath: str, content: str, metadata: Optional[Dict] = None):
        """
        Write content to file.
        
        Args:
            filepath: Path to output file
            content: Content to write
            metadata: Optional metadata to include
        """
        path = Path(filepath)
        path.parent.mkdir(parents=True, exist_ok=True)
        
        try:
            with open(path, 'w', encoding='utf-8') as f:
                # Add metadata as comment if provided
                if metadata:
                    f.write(f"<!-- Generated: {datetime.now().isoformat()} -->\n")
                    f.write(f"<!-- Model: {metadata.get('model', 'N/A')} -->\n")
                    f.write(f"<!-- Tokens: {metadata.get('total_tokens', 'N/A')} -->\n")
                    f.write(f"<!-- Cost: ${metadata.get('cost', 0):.6f} -->\n\n")
                
                f.write(content)
            
            console.print(f"[green]✓[/green] Wrote summary to {filepath}")
            
        except Exception as e:
            console.print(f"[red]✗[/red] Error writing file: {e}")
            raise
    
    def read_json(self, filepath: str) -> Dict:
        """Read JSON file."""
        with open(filepath, 'r') as f:
            return json.load(f)
    
    def write_json(self, filepath: str, data: Dict):
        """Write JSON file."""
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)

# Test file handler
file_handler = FileHandler()

# Create a test file
test_content = """
Artificial intelligence is rapidly transforming industries worldwide.
From healthcare to finance, AI systems are being deployed to solve complex problems.
Machine learning algorithms can analyze vast amounts of data to identify patterns
and make predictions that would be impossible for humans to achieve manually.
"""

# Write test file
test_file = "test_input.txt"
with open(test_file, 'w') as f:
    f.write(test_content)

# Read file
content = file_handler.read_file(test_file)
console.print(f"Read {len(content)} characters")

# Summarize and save
result = summarizer.summarize(content, mode="concise")
file_handler.write_file("test_output.md", result["summary"], metadata=result)

# Clean up
os.remove(test_file)
console.print("[green]✓[/green] File operations test complete")

## Part 6: Command-Line Interface

Build a user-friendly CLI:

In [ ]:
class SummarizerCLI:
    """
    Command-line interface for the summarizer.
    """
    
    def __init__(self):
        self.config = Config()
        self.summarizer = SummarizerEngine(self.config)
        self.file_handler = FileHandler()
    
    def display_welcome(self):
        """Display welcome message."""
        console.print(Panel.fit(
            "[bold blue]Text Summarizer AI[/bold blue]\n"
            "[dim]Provided by: ADC ENGINEERING & CONSULTING LTD[/dim]",
            border_style="blue"
        ))
    
    def get_text_input(self) -> str:
        """Get text from user."""
        console.print("\n[cyan]Enter text to summarize (press Ctrl+D or Ctrl+Z when done):[/cyan]")
        lines = []
        
        try:
            while True:
                line = input()
                lines.append(line)
        except EOFError:
            pass
        
        return "\n".join(lines)
    
    def select_mode(self) -> str:
        """Let user select summarization mode."""
        modes = ["concise", "detailed", "bullets", "key_points", "executive", "technical"]
        
        console.print("\n[cyan]Select summarization mode:[/cyan]")
        for i, mode in enumerate(modes, 1):
            console.print(f"  {i}. {mode}")
        
        while True:
            try:
                choice = int(input("\nEnter number (1-6): "))
                if 1 <= choice <= len(modes):
                    return modes[choice - 1]
            except (ValueError, EOFError):
                pass
            console.print("[red]Invalid choice. Please try again.[/red]")
    
    def summarize_interactive(self):
        """Interactive summarization."""
        # Get input source
        console.print("\n[cyan]Input source:[/cyan]")
        console.print("  1. Enter text directly")
        console.print("  2. Read from file")
        
        try:
            choice = int(input("\nEnter number (1-2): "))
        except (ValueError, EOFError):
            console.print("[red]Invalid input.[/red]")
            return
        
        # Get text
        if choice == 1:
            text = self.get_text_input()
        elif choice == 2:
            filepath = input("Enter file path: ").strip()
            try:
                text = self.file_handler.read_file(filepath)
            except Exception as e:
                console.print(f"[red]Error: {e}[/red]")
                return
        else:
            console.print("[red]Invalid choice.[/red]")
            return
        
        if not text.strip():
            console.print("[red]No text provided.[/red]")
            return
        
        # Select mode
        mode = self.select_mode()
        
        # Summarize with progress indicator
        console.print("\n[yellow]Generating summary...[/yellow]")
        
        with Progress(
            SpinnerColumn(),
            TextColumn("[progress.description]{task.description}"),
            console=console
        ) as progress:
            task = progress.add_task("Summarizing...", total=None)
            result = self.summarizer.summarize(text, mode=mode)
        
        # Display result
        console.print("\n")
        console.print(Panel(
            result["summary"],
            title=f"[bold]{mode.upper()} Summary[/bold]",
            border_style="green"
        ))
        
        # Display metadata
        console.print(f"\n[dim]Model: {result['model']}[/dim]")
        console.print(f"[dim]Tokens: {result['total_tokens']} | Cost: ${result['cost']:.6f}[/dim]")
        
        # Ask to save
        save = input("\nSave summary to file? (y/n): ").strip().lower()
        if save == 'y':
            output_file = input("Enter output filename: ").strip()
            if output_file:
                self.file_handler.write_file(output_file, result["summary"], metadata=result)
    
    def run(self):
        """Run the CLI application."""
        self.display_welcome()
        
        while True:
            console.print("\n[cyan]Main Menu:[/cyan]")
            console.print("  1. Summarize text")
            console.print("  2. View configuration")
            console.print("  3. Exit")
            
            try:
                choice = input("\nEnter choice: ").strip()
            except EOFError:
                break
            
            if choice == "1":
                self.summarize_interactive()
            elif choice == "2":
                self.config.display()
            elif choice == "3":
                console.print("\n[green]Thank you for using Text Summarizer AI![/green]")
                break
            else:
                console.print("[red]Invalid choice.[/red]")

# Run the CLI (uncomment to test)
# cli = SummarizerCLI()
# cli.run()

### Exercise 6.1: Enhanced CLI Features

Add these features to the CLI:
1. Command-line arguments support (using `click` or `argparse`)
2. Batch file processing
3. Configuration file support
4. History of summaries
5. Export to different formats (PDF, JSON, HTML)

In [ ]:
# TODO: Implement enhanced CLI with click

import click

@click.group()
@click.version_option(version='1.0.0')
def cli():
    """Text Summarizer AI - Command Line Interface"""
    pass

@cli.command()
@click.argument('input_file', type=click.Path(exists=True))
@click.option('--mode', '-m', default='concise', 
              type=click.Choice(['concise', 'detailed', 'bullets', 'key_points']))
@click.option('--output', '-o', type=click.Path(), help='Output file path')
@click.option('--stream/--no-stream', default=False, help='Enable streaming output')
def summarize(input_file, mode, output, stream):
    """Summarize text from INPUT_FILE."""
    # TODO: Implement
    pass

@cli.command()
@click.argument('directory', type=click.Path(exists=True))
@click.option('--mode', '-m', default='concise')
@click.option('--pattern', '-p', default='*.txt', help='File pattern to match')
def batch(directory, mode, pattern):
    """Batch summarize files in DIRECTORY."""
    # TODO: Implement
    pass

@cli.command()
def config():
    """Display current configuration."""
    # TODO: Implement
    pass

# Run CLI
# if __name__ == '__main__':
#     cli()

## Part 7: Logging and Monitoring

Add comprehensive logging:

In [ ]:
import logging
from datetime import datetime

class AppLogger:
    """
    Application logger with file and console output.
    """
    
    def __init__(self, name: str = "summarizer", log_file: Optional[str] = None):
        """
        Initialize logger.
        
        Args:
            name: Logger name
            log_file: Path to log file (optional)
        """
        self.logger = logging.getLogger(name)
        self.logger.setLevel(logging.DEBUG)
        
        # Create formatters
        detailed_formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        
        simple_formatter = logging.Formatter(
            '%(levelname)s: %(message)s'
        )
        
        # Console handler
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        console_handler.setFormatter(simple_formatter)
        self.logger.addHandler(console_handler)
        
        # File handler (if specified)
        if log_file:
            file_handler = logging.FileHandler(log_file)
            file_handler.setLevel(logging.DEBUG)
            file_handler.setFormatter(detailed_formatter)
            self.logger.addHandler(file_handler)
    
    def log_operation(self, operation: str, details: Dict):
        """Log an operation with details."""
        self.logger.info(f"Operation: {operation}")
        self.logger.debug(f"Details: {json.dumps(details, indent=2)}")
    
    def log_error(self, error: Exception, context: str = ""):
        """Log an error."""
        self.logger.error(f"Error in {context}: {str(error)}", exc_info=True)
    
    def log_summary(self, result: Dict):
        """Log a summarization operation."""
        self.logger.info(
            f"Summary generated - Mode: {result['mode']}, "
            f"Tokens: {result['total_tokens']}, "
            f"Cost: ${result['cost']:.6f}"
        )

# Test logger
logger = AppLogger("test_logger", log_file="summarizer.log")
logger.log_operation("test_operation", {"key": "value"})
logger.log_summary({
    "mode": "concise",
    "total_tokens": 150,
    "cost": 0.0005
})

console.print("[green]✓[/green] Logging configured")

## Part 8: Complete Application

Integrate all components into a production-ready application:

In [ ]:
class TextSummarizerApp:
    """
    Complete text summarizer application.
    Integrates all components: config, summarizer, file handling, logging.
    """
    
    def __init__(self, config_file: Optional[str] = None):
        """
        Initialize the application.
        
        Args:
            config_file: Path to configuration file (optional)
        """
        # Load configuration
        self.config = Config(config_file)
        
        # Initialize components
        self.summarizer = SummarizerEngine(self.config)
        self.file_handler = FileHandler()
        self.logger = AppLogger(
            "summarizer_app",
            log_file="summarizer_app.log"
        )
        
        # Track session statistics
        self.session_stats = {
            "summaries_generated": 0,
            "total_tokens": 0,
            "total_cost": 0.0,
            "start_time": datetime.now()
        }
        
        self.logger.log_operation("app_initialized", {
            "config": self.config.config,
            "timestamp": datetime.now().isoformat()
        })
    
    def summarize_text(
        self,
        text: str,
        mode: str = "concise",
        stream: bool = False
    ) -> Dict:
        """
        Summarize text with full error handling and logging.
        """
        try:
            if stream:
                return self._summarize_stream(text, mode)
            else:
                return self._summarize_normal(text, mode)
        except Exception as e:
            self.logger.log_error(e, "summarize_text")
            raise
    
    def _summarize_normal(self, text: str, mode: str) -> Dict:
        """Non-streaming summarization."""
        result = self.summarizer.summarize(text, mode=mode)
        
        # Update statistics
        self.session_stats["summaries_generated"] += 1
        self.session_stats["total_tokens"] += result["total_tokens"]
        self.session_stats["total_cost"] += result["cost"]
        
        # Log operation
        self.logger.log_summary(result)
        
        return result
    
    def _summarize_stream(self, text: str, mode: str) -> Dict:
        """Streaming summarization."""
        console.print("\n[bold]Summary:[/bold]")
        console.print("-" * 80)
        
        metadata = None
        for chunk in self.summarizer.summarize_stream(text, mode=mode):
            if isinstance(chunk, dict):
                metadata = chunk
            else:
                console.print(chunk, end="", markup=False)
        
        console.print("\n" + "-" * 80)
        
        # Update statistics
        if metadata:
            self.session_stats["summaries_generated"] += 1
            self.session_stats["total_tokens"] += metadata["total_tokens"]
            self.session_stats["total_cost"] += metadata["cost"]
            
            self.logger.log_summary(metadata)
        
        return metadata or {}
    
    def summarize_file(
        self,
        input_file: str,
        output_file: Optional[str] = None,
        mode: str = "concise",
        stream: bool = False
    ) -> Dict:
        """
        Summarize content from file.
        """
        # Read file
        text = self.file_handler.read_file(input_file)
        
        # Summarize
        result = self.summarize_text(text, mode=mode, stream=stream)
        
        # Save if output specified
        if output_file:
            self.file_handler.write_file(
                output_file,
                result["summary"],
                metadata=result
            )
        
        return result
    
    def get_session_report(self) -> Dict:
        """Get session statistics."""
        duration = datetime.now() - self.session_stats["start_time"]
        
        return {
            **self.session_stats,
            "duration_seconds": duration.total_seconds(),
            "avg_tokens_per_summary": (
                self.session_stats["total_tokens"] / self.session_stats["summaries_generated"]
                if self.session_stats["summaries_generated"] > 0 else 0
            ),
            "avg_cost_per_summary": (
                self.session_stats["total_cost"] / self.session_stats["summaries_generated"]
                if self.session_stats["summaries_generated"] > 0 else 0
            )
        }
    
    def display_session_report(self):
        """Display session statistics."""
        report = self.get_session_report()
        
        table = Table(title="Session Report")
        table.add_column("Metric", style="cyan")
        table.add_column("Value", style="green")
        
        table.add_row("Summaries Generated", str(report["summaries_generated"]))
        table.add_row("Total Tokens", str(report["total_tokens"]))
        table.add_row("Total Cost", f"${report['total_cost']:.6f}")
        table.add_row("Session Duration", f"{report['duration_seconds']:.1f}s")
        table.add_row("Avg Tokens/Summary", f"{report['avg_tokens_per_summary']:.0f}")
        table.add_row("Avg Cost/Summary", f"${report['avg_cost_per_summary']:.6f}")
        
        console.print("\n")
        console.print(table)

# Test the complete application
app = TextSummarizerApp()

console.print("\n[bold]Testing Complete Application[/bold]\n")

# Test text summarization
test_text = """
Cloud computing has revolutionized how businesses deploy and manage their IT infrastructure.
Instead of maintaining physical servers, companies can now rent computing resources on-demand
from cloud providers like AWS, Azure, and Google Cloud. This shift offers numerous benefits
including cost savings, scalability, reliability, and global reach. Organizations can quickly
scale resources up or down based on demand, paying only for what they use. Additionally,
cloud providers handle infrastructure maintenance, security updates, and disaster recovery,
allowing companies to focus on their core business objectives.
"""

result = app.summarize_text(test_text, mode="bullets")

console.print(Panel(
    result["summary"],
    title="Summary Result",
    border_style="green"
))

# Display session report
app.display_session_report()

## Challenge Projects

### Challenge 1: Multi-Document Summarizer

Build a system that:
- Summarizes multiple related documents
- Identifies common themes across documents
- Generates a unified summary
- Creates a comparison matrix

In [ ]:
class MultiDocumentSummarizer:
    """
    Summarize multiple documents and find common themes.
    
    TODO: Implement:
    1. Load multiple documents
    2. Generate individual summaries
    3. Extract common themes
    4. Create unified summary
    5. Generate comparison matrix
    6. Visualize relationships
    """
    
    def __init__(self, app: TextSummarizerApp):
        self.app = app
        self.documents = []
        self.summaries = []
    
    # TODO: Implement methods
    
    pass

### Challenge 2: Meeting Transcript Summarizer

Build a specialized tool for meeting transcripts:
- Identify speakers and their main points
- Extract action items and decisions
- Generate meeting minutes
- Create follow-up task list

In [ ]:
class MeetingSummarizer:
    """
    Specialized summarizer for meeting transcripts.
    
    TODO: Implement:
    1. Parse transcript with speaker labels
    2. Extract key discussion points per speaker
    3. Identify action items
    4. Generate structured meeting minutes
    5. Create task list with assignments
    """
    
    def __init__(self, app: TextSummarizerApp):
        self.app = app
    
    # TODO: Implement methods
    
    pass

### Challenge 3: Research Paper Summarizer

Create a tool for academic papers:
- Extract abstract, methodology, results, conclusions
- Identify key findings and contributions
- Generate citation-ready summaries
- Compare with related work

In [ ]:
class ResearchPaperSummarizer:
    """
    Specialized summarizer for research papers.
    
    TODO: Implement:
    1. Parse paper structure (abstract, intro, methods, results, conclusion)
    2. Extract key findings
    3. Identify contributions
    4. Generate structured summary
    5. Create citation-ready format
    """
    
    def __init__(self, app: TextSummarizerApp):
        self.app = app
    
    # TODO: Implement methods
    
    pass

## Summary

In this lab, you've built a complete, production-ready AI application with:

1. ✅ Configuration management
2. ✅ Robust error handling with retry logic
3. ✅ Core summarization engine with multiple modes
4. ✅ File input/output support
5. ✅ Command-line interface
6. ✅ Logging and monitoring
7. ✅ Streaming responses
8. ✅ Session statistics tracking

### Key Achievements

- **Modular Design**: Clean separation of concerns
- **Error Resilience**: Comprehensive error handling
- **User Experience**: Rich CLI with progress indicators
- **Observability**: Logging and statistics
- **Production Ready**: Configuration, monitoring, and best practices

### Best Practices Demonstrated

1. **Configuration Management**: Centralized, file-based configuration
2. **Error Handling**: Exponential backoff, graceful degradation
3. **Logging**: Structured logging with multiple outputs
4. **Code Organization**: Clear class hierarchy and responsibilities
5. **User Experience**: Rich visual feedback and streaming
6. **Cost Awareness**: Token counting and cost estimation
7. **File Operations**: Safe file handling with validation

### Next Steps

1. Complete the challenge projects
2. Add authentication and user management
3. Build a web interface (Flask/FastAPI)
4. Add database for storing summaries
5. Implement caching for repeated requests
6. Add support for more file formats
7. Deploy to production environment

**Congratulations!** You've completed Week 1 and built your first complete AI application!

**Provided by:** ADC ENGINEERING & CONSULTING LTD